In [35]:
import os
import re

# 输入和输出文件夹
input_folder = "MDA训练集"  # 存放原始 txt 文件的文件夹
output_folder = "output_txt"  # 存放处理后 txt 文件的文件夹

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 识别表格及相关行的正则表达式
table_patterns = [
    # 匹配包含数字、百分比、多个空格的表格数据行
    re.compile(r'^\s*.*(\d{1,3}(,\d{3})*\.\d+|\d+\.\d+%|\s{2,}).*$', re.MULTILINE),
    # 匹配“单位：”的标题行
    re.compile(r'^\s*单位：.*$', re.MULTILINE),
    # 匹配分隔线
    re.compile(r'^\s*[-|]+.*$', re.MULTILINE),
    # 匹配表格标题行（包含年份或“变动比例”）
    re.compile(r'^\s*.*(20\d{2}年|\b变动比例\b).*$', re.MULTILINE),
    
]

def is_table_related_line(line):
    """
    判断一行是否与表格相关（数据行、标题行、描述行）
    """
    for pattern in table_patterns:
        if pattern.match(line):
            return True
    return False

def remove_tables_and_related(text):
    """
    删除表格及其相关内容（数据、标题、描述）
    """
    # 按行处理
    lines = text.splitlines()
    cleaned_lines = [line for line in lines if not is_table_related_line(line)]
    # 合并行，清理多余空行
    cleaned_text = '\n'.join(cleaned_lines)
    cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text).strip()
    return cleaned_text

# 遍历输入文件夹中的所有 txt 文件
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_path = os.path.join(input_folder, filename)
        # 生成输出文件名，添加 _notable 后缀
        base_name = os.path.splitext(filename)[0]  # 获取文件名（不含扩展名）
        output_filename = f"{base_name}_notable.txt"  # 添加 _notable 后缀
        output_path = os.path.join(output_folder, output_filename)
        
        # 读取文件内容
        with open(input_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        # 去除表格及相关内容
        cleaned_content = remove_tables_and_related(content)
        
        # 保存到新的文件
        with open(output_path, 'w', encoding='utf-8') as file:
            file.write(cleaned_content)
        
        print(f"已处理: {filename} -> {output_filename}")

print("所有文件处理完成！")

已处理: 895-双汇发展-2021.txt -> 895-双汇发展-2021_notable.txt
已处理: 2475-立讯精密-2021.txt -> 2475-立讯精密-2021_notable.txt
已处理: 600566-济川药业-2021.txt -> 600566-济川药业-2021_notable.txt
已处理: 963-华东医药-2021.txt -> 963-华东医药-2021_notable.txt
已处理: 2429-兆驰股份-2021.txt -> 2429-兆驰股份-2021_notable.txt
已处理: 2552-宝鼎科技-2021.txt -> 2552-宝鼎科技-2021_notable.txt
已处理: 600031-三一重工-2021.txt -> 600031-三一重工-2021_notable.txt
已处理: 2415-海康威视-2021.txt -> 2415-海康威视-2021_notable.txt
已处理: 300339-润和软件-2021.txt -> 300339-润和软件-2021_notable.txt
已处理: 2241-歌尔股份-2021.txt -> 2241-歌尔股份-2021_notable.txt
已处理: 600276-恒瑞医药-2021.txt -> 600276-恒瑞医药-2021_notable.txt
已处理: 425-徐工机械-2021.txt -> 425-徐工机械-2021_notable.txt
已处理: 2507-涪陵榨菜-2021.txt -> 2507-涪陵榨菜-2021_notable.txt
已处理: 600519-贵州茅台-2021.txt -> 600519-贵州茅台-2021_notable.txt
已处理: 2230-科大讯飞-2021.txt -> 2230-科大讯飞-2021_notable.txt
所有文件处理完成！


In [37]:
import os
import re

input_folder = "MDA训练集"
output_folder = "output_txt"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def is_table_like(line):
    # 特征1：出现至少2个空格或tab对齐
    if len(re.findall(r'[\t ]{2,}', line)) >= 1:
        return True

    # 特征2：包含3个以上数字（不要求格式）
    if len(re.findall(r'\d+', line)) >= 3:
        return True

    # 特征3：含有表格相关字段词
    if re.search(r'(金额|比例|占比|资产|负债|说明|人数|本期|上期|变动)', line):
        return True

    # 特征4：以多字段构成，列数多
    columns = re.split(r'[\t ]{2,}', line.strip())
    if len(columns) >= 3:
        return True

    return False

def remove_table_blocks(text):
    lines = text.splitlines()
    n = len(lines)
    to_delete = set()
    i = 0

    while i < n:
        if is_table_like(lines[i]):
            # 开始识别表格块
            start = i
            end = i

            while start > 0 and is_table_like(lines[start - 1]):
                start -= 1
            while end + 1 < n and is_table_like(lines[end + 1]):
                end += 1

            for j in range(start, end + 1):
                to_delete.add(j)

            i = end + 1  # 跳过这段
        else:
            i += 1

    # 清除标记的行
    cleaned = [line for idx, line in enumerate(lines) if idx not in to_delete]
    cleaned_text = '\n'.join(cleaned)
    cleaned_text = re.sub(r'\n\s*\n+', '\n', cleaned_text).strip()
    return cleaned_text

# 主处理流程
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_path = os.path.join(input_folder, filename)
        base_name = os.path.splitext(filename)[0]
        output_filename = f"{base_name}_notable.txt"
        output_path = os.path.join(output_folder, output_filename)

        with open(input_path, 'r', encoding='utf-8') as f:
            content = f.read()

        cleaned = remove_table_blocks(content)

        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(cleaned)

        print(f"已处理: {filename} -> {output_filename}")

print("所有文件处理完成！")

已处理: 895-双汇发展-2021.txt -> 895-双汇发展-2021_notable.txt
已处理: 2475-立讯精密-2021.txt -> 2475-立讯精密-2021_notable.txt
已处理: 600566-济川药业-2021.txt -> 600566-济川药业-2021_notable.txt
已处理: 963-华东医药-2021.txt -> 963-华东医药-2021_notable.txt
已处理: 2429-兆驰股份-2021.txt -> 2429-兆驰股份-2021_notable.txt
已处理: 2552-宝鼎科技-2021.txt -> 2552-宝鼎科技-2021_notable.txt
已处理: 600031-三一重工-2021.txt -> 600031-三一重工-2021_notable.txt
已处理: 2415-海康威视-2021.txt -> 2415-海康威视-2021_notable.txt
已处理: 300339-润和软件-2021.txt -> 300339-润和软件-2021_notable.txt
已处理: 2241-歌尔股份-2021.txt -> 2241-歌尔股份-2021_notable.txt
已处理: 600276-恒瑞医药-2021.txt -> 600276-恒瑞医药-2021_notable.txt
已处理: 425-徐工机械-2021.txt -> 425-徐工机械-2021_notable.txt
已处理: 2507-涪陵榨菜-2021.txt -> 2507-涪陵榨菜-2021_notable.txt
已处理: 600519-贵州茅台-2021.txt -> 600519-贵州茅台-2021_notable.txt
已处理: 2230-科大讯飞-2021.txt -> 2230-科大讯飞-2021_notable.txt
所有文件处理完成！
